## Step 0: Install docker and place sudo password in ~/.sudopassword

### Removed previous run containers (Skip when starting for first time)

In [21]:
!sudo -S docker rm /zookeeper-server < ~/.sudopassword
!sudo -S docker rm /kafka-server < ~/.sudopassword
!sudo -S docker rm /event-stream < ~/.sudopassword
!sudo -S docker network rm app-tier < ~/.sudopassword

[sudo] password for kalinga: /zookeeper-server
[sudo] password for kalinga: /kafka-server
[sudo] password for kalinga: /event-stream
[sudo] password for kalinga: app-tier


## Step 1: Create a network

In [22]:
!sudo -S docker network create app-tier --driver bridge < ~/.sudopassword

[sudo] password for kalinga: 2c23082b1f62b4573c63967bf5a1237d4b4996da7723ff033272af4bc081cea0


## Step 2: Launch the Zookeeper server instance

In [23]:
!sudo -S  docker run -d --name zookeeper-server --network app-tier -e ALLOW_ANONYMOUS_LOGIN=yes bitnami/zookeeper:latest < ~/.sudopassword

[sudo] password for kalinga: ae3034974523148932d5b4f3c3108b09f4d66a55bc5f65df4c1a7ce6d3a70f63


## Step 3: Launch the Kafka server instance

In [24]:
!sudo -S docker run -d --name kafka-server \
    --network app-tier \
    -e ALLOW_PLAINTEXT_LISTENER=yes \
    -e KAFKA_CFG_ZOOKEEPER_CONNECT=zookeeper-server:2181 \
    bitnami/kafka:latest < ~/.sudopassword

[sudo] password for kalinga: 315d2f8ea9fc9c46fa275f1b79eaf56b5bd5a9924ef7c7e78eccf7a8f0c392ef


## Step 4: Launch event-stream  instance

In [ ]:
extract the zip and get inside the extracted folder  


In [25]:
!sudo -S docker build -t kalinga/event-stream:v0.1 . < ~/.sudopassword

[sudo] password for kalinga: Sending build context to Docker daemon  62.46kB
Step 1/8 : FROM python:3
 ---> b162b5322f1c
Step 2/8 : LABEL Author="Kalinga Bhusan Ray"
 ---> Using cache
 ---> 55e95b0d90db
Step 3/8 : LABEL E-Mail="kalinga-bhusan.ray@tu-ilmenau.de"
 ---> Using cache
 ---> 5e490150ac8b
Step 4/8 : LABEL Desc="This is an image for event stream engiene"
 ---> Using cache
 ---> 430a9dc7ecdd
Step 5/8 : RUN apt-get update &&     apt-get install -y vim
 ---> Using cache
 ---> 63f97c80a8c5
Step 6/8 : ADD process_event_stream.py /
 ---> 5fe68cdbb021
Step 7/8 : RUN pip install faust typing_extensions
 ---> Running in cf2765fe7e84
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp38-none-any.whl size=15355 sha256=44cd5e53ac07bbc8fef4b86a7d911f0179674518d38dd0eeb63830fe5f3bfccc
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
  Created wheel for colorclass: filename=colorclass-2.2.0-cp38-none-any.whl size=19398

In [26]:
!sudo -S docker run -d \
                -it  \
                --name event-stream \
                --mount type=bind,source="$(pwd)"/.,target=/app \
                --network app-tier \
                kalinga/event-stream:v0.1 /bin/bash < ~/.sudopassword    

[sudo] password for kalinga: 629d5f35935b09b0dfd05106643ef89a782a6d8273eeff626aeeb99728a6c197


In [27]:
!sudo -S docker ps  < ~/.sudopassword

[sudo] password for kalinga: CONTAINER ID        IMAGE                       COMMAND                  CREATED              STATUS                  PORTS                                    NAMES
629d5f35935b        kalinga/event-stream:v0.1   "/bin/bash"              5 seconds ago        Up Less than a second   80/tcp                                   event-stream
315d2f8ea9fc        bitnami/kafka:latest        "/entrypoint.sh /run…"   About a minute ago   Up About a minute       9092/tcp                                 kafka-server
ae3034974523        bitnami/zookeeper:latest    "/entrypoint.sh /run…"   About a minute ago   Up About a minute       2181/tcp, 2888/tcp, 3888/tcp, 8080/tcp   zookeeper-server


In [30]:
!ls

containerFS	    operations		     README.md		    ut-acc-init
Dockerfile	    process_event_stream.py  stream-event-app-data  ut-order
event-stream.ipynb  __pycache__		     test.py


### For existing containers (During development after restarting the host machine)

In [1]:
!sudo -S docker start /zookeeper-server < ~/.sudopassword
!sudo -S docker start /kafka-server < ~/.sudopassword
!sudo -S docker start /event-stream < ~/.sudopassword

[sudo] password for kalinga: /zookeeper-server
[sudo] password for kalinga: /kafka-server
[sudo] password for kalinga: /event-stream


In [3]:
!sudo -S docker ps < ~/.sudopassword

[sudo] password for kalinga: CONTAINER ID        IMAGE                       COMMAND                  CREATED             STATUS              PORTS                                    NAMES
f1966e227cf6        kalinga/event-stream:v0.1   "/bin/bash"              31 hours ago        Up 15 seconds       80/tcp                                   event-stream
91792eec8d10        bitnami/kafka:latest        "/entrypoint.sh /run…"   31 hours ago        Up 24 seconds       9092/tcp                                 kafka-server
27f4dbfafd0d        bitnami/zookeeper:latest    "/entrypoint.sh /run…"   31 hours ago        Up 28 seconds       2181/tcp, 2888/tcp, 3888/tcp, 8080/tcp   zookeeper-server


### Step 5: Copy the Unit tests

In [28]:
!sudo -S docker cp  ut-acc-init kafka-server:/ < ~/.sudopassword

[sudo] password for kalinga: 

In [29]:
!sudo -S docker cp  ut-order kafka-server:/ < ~/.sudopassword

[sudo] password for kalinga: 

### Step 6: Start the app and test

#### [In separate terminals]
docker exec -it event-stream bash  
cd app  
faust -A process_event_stream worker -l info  

docker exec -it kafka-server bash  

[kafka-server terminal]  
kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic transaction --producer.config /opt/bitnami/kafka/conf/producer.properties <  ut-acc-init  
kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic transaction --producer.config /opt/bitnami/kafka/conf/producer.properties <  ut-order   

### Step 6: Extend the requirements

Not yet implemented:  
    `high-frequency-small-interval`  
    `doubled-transaction`


#### Development shortcuts

In [7]:
!sudo -S docker run -d \
                -it  \
                --name event-stream \
                --mount type=bind,source="$(pwd)"/.,target=/app \
                --network app-tier \
                kalinga/event-stream:v0.1 /bin/bash < ~/.sudopassword    

[sudo] password for kalinga: db971051eb54966f51e53d7a1759410614a1f8cc75983b362ee06768ae146a34


In [4]:
#!sudo -S docker container ls --all < ~/.sudopassword

In [ ]:
!sudo -S docker cp event-stream:/process_event_stream.py .  < ~/.sudopassword

In [ ]:
!sudo -S chown kalinga:kalinga process_event_stream.py < ~/.sudopassword

In [2]:
!sudo -S docker stop event-stream < ~/.sudopassword

[sudo] password for kalinga: event-stream


In [6]:
!sudo -S docker rm /event-stream < ~/.sudopassword

[sudo] password for kalinga: /event-stream


In [20]:
!sudo -S docker stop /zookeeper-server < ~/.sudopassword
!sudo -S docker stop /kafka-server < ~/.sudopassword
!sudo -S docker stop /event-stream < ~/.sudopassword

[sudo] password for kalinga: /zookeeper-server
[sudo] password for kalinga: /kafka-server
[sudo] password for kalinga: /event-stream
